# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
print("\nTarget (y):")
print(y.head())


Target (y):
0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
print("Features (X):")
X.head()

Features (X):


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training and testing dataset
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_test_scaled[:5]

array([[ 0.27988214, -0.67947695,  1.87699031, -0.24897225,  0.11388606,
        -0.64671182, -0.11681233,  0.96943574,  1.43482818,  0.00384497,
         0.05226031],
       [ 0.22360652,  0.55286199,  0.05722821,  0.10845901,  0.17694246,
         0.11885252,  0.72287966,  0.31294474, -0.35064271, -0.17390392,
        -0.22508797],
       [ 1.18029216, -1.07158479,  1.57369663,  0.82332153,  1.81640878,
        -0.93379844, -0.92540463,  0.13579637, -0.79701044,  0.2408435 ,
         0.79185571],
       [-0.67680351, -0.39939992, -0.34716337, -0.46343101, -0.22241472,
         1.17150347,  0.25638411, -0.59363809,  0.35079228, -0.11465429,
        -0.04018912],
       [-0.67680351, -0.79150776, -0.75155495, -0.3204585 , -0.47464031,
        -0.16823411, -0.676607  , -2.19839389, -0.09557544, -0.76640023,
         2.64084423]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [14]:
# Define the the number of inputs (features) to the model
num_features = X_train_scaled.shape[1]

# Review the number of features
print("Number of features:", num_features)

Number of features: 11


In [15]:
# Define the number of hidden nodes for the first hidden layer
num_hidden_nodes_layer1 = 8 

# Define the number of hidden nodes for the second hidden layer
num_hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
num_output_neurons = 1 

In [16]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(num_hidden_nodes_layer1, input_shape=(num_features,), activation='relu'))

# Add the second hidden layer
model.add(Dense(num_hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(num_output_neurons, activation='sigmoid'))

C:\Users\Kevin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [18]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [19]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.5418 - loss: 0.7483 
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.5398 - loss: 0.7057
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - accuracy: 0.5792 - loss: 0.6868
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.6120 - loss: 0.6678
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.6151 - loss: 0.6580
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.6595 - loss: 0.6317
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.6816 - loss: 0.6022
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.6884 - loss: 0.5951
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.6875 - loss: 0.5846
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - accuracy: 0.6946 - loss: 0.5775
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7130 - loss: 0.5652
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [20]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_metric = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Test Loss: {test_loss}")
print(f"Test Metric: {test_metric}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.7900 - loss: 0.4951
Test Loss: 0.5218132138252258
Test Metric: 0.762499988079071


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [21]:
# Set the model's file path
path = 'student_loans.keras'

# Export your model to a keras file
model.save(path)
print("Model saved to 'student_loans.keras'")

Model saved to 'student_loans.keras'


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [22]:
from tensorflow.keras.models import load_model

# Set the model's file path
path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model('student_loans.keras')

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [23]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(f"Test Loss: {test_loss}")
print(f"Test Metric: {test_metric}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step
Test Loss: 0.5218132138252258
Test Metric: 0.762499988079071


In [28]:
import numpy as np

# Save the predictions to a DataFrame and round the predictions to binary results
binary_predictions = np.round(predictions).astype(int)
df_predictions = pd.DataFrame({
    'Prediction': binary_predictions.flatten()  # Flatten to convert from array to series
})
df_predictions.head()

,Prediction
0,0
1,0
2,1
3,1
4,1


### Step 4: Display a classification report with the y test data and predictions

In [29]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, binary_predictions, target_names=['Class 0', 'Class 1'])
print(report)

              precision    recall  f1-score   support

     Class 0       0.74      0.77      0.76       154
     Class 1       0.78      0.75      0.77       166

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



---
## Discuss creating a recommendation system for student loans

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**




1. The relevant data includes student profile data (demographics, academic information, financial status, and loan history), loan product data (loan amounts, interest rates, repayment terms, eligibility criteria, and additional benefits), behavioral data (loan search history, interaction data, and feedback), and external factors (economic indicators and regulatory changes). This data would be useful for creating recommendations for individual student needs and current market conditions.

2. The system would use content-based filtering as it matches student profiles with loan attributes. Detailed data on student profiles and loan products allows for precise, personalized recommendations based on individual needs and preferences.

3. One challenge is ensuring the protection of sensitive personal and financial data to comply with privacy regulations and prevent breaches. Another challenge is avoiding bias in recommendations to ensure equal access to loan options for all students, preventing financial inequalities. Our recommendations should be free of bias that could result in enequitable outcomes for different demographics.